# Exploring Toronto
## Part 1 - Scraping Wiki and Creating the datatframe

In [1]:
#install beautifulsoup4, parser, request, uncomment and run if needed
#!pip install beautifulsoup4
#!pip install lxml
#!pip install requests

#import libs
import pandas as pd
import requests
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [2]:
Page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(Page.content,'lxml')
table = soup.find_all('table')[0] 
table_rows = table.find_all('tr')

#create a dictionary of all the rows
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        data.append(row)

#create dataframe with data dictionary
df = pd.DataFrame(data, columns=["Postal Code", "Borough", "Neighborhood"])

### The initial dataframe
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
df.head()

### Removing 'Not assigned' Boroughs
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
# recreate dataframe where Borough not equals 'Not assigned'
df=df[df.Borough != 'Not assigned']
df.head(8)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue


### Replacing 'Not assigned' Neighborhood with Borough
If a cell has a borough but a Not assigned Neighborhoods, then the Neighborhoods will be the same as the borough. 

In [4]:
# Replace Neighborhood with Borough where Neighborhood = 'Not assigned'
for index, row in df.iterrows():
    if row['Neighborhood']=='Not assigned':
        row['Neighborhood']= row['Borough']
df.head(8)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue


### Combine Neighborhoods with Common Postal Codes
More than one Neighborhoods can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two Neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the Neighborhoods separated with a comma.

In [5]:
#set postal code and Borough as the index
df.set_index(['Postal Code','Borough'],inplace=True)

#Combine Rows of Multiindex DataFrame into Comma Separated Lists
df = df.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)

In [6]:
#reset the index
df1=df.reset_index() 
df1.head(8)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North


### Final Dataframe Shape

In [7]:
df1.shape

(103, 3)

## Part 2 - Retrieving Geographical Coordinates

In [8]:
#import Geospatial_Coordinates.csv into a dataframe
df_pc= pd.read_csv('Documents/Coursera - IBM data science/capstone/Geospatial_Coordinates.csv')
df_pc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
#set postal code as the index
df1.set_index(['Postal Code'],inplace=True)
df_pc.set_index (['Postal Code'],inplace=True)

#combined df and df_pc based on PostalCode index
df2 = pd.concat([df1, df_pc], axis=1, join='inner')

#reset index
df3=df2.reset_index() 
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
